<a href="https://colab.research.google.com/github/stefanoBam/Admission_Algorithm/blob/main/Admission_Algorithm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Pull Files from Github

In [8]:
!git clone https://github.com/stefanoBam/Admission_Algorithm.git

#!git clone https://github.com/facebookresearch/llama.git

fatal: destination path 'Admission_Algorithm' already exists and is not an empty directory.
Cloning into 'llama'...
remote: Enumerating objects: 451, done.
remote: Counting objects: 100% (34/34), done.
remote: Compressing objects: 100% (22/22), done.
remote: Total 451 (delta 11), reused 26 (delta 11), pack-reused 417
Receiving objects: 100% (451/451), 1.11 MiB | 3.71 MiB/s, done.
Resolving deltas: 100% (229/229), done.


# Functions

In [1]:
## Import and intitialize LLM
## WARNING takes a while to download a 750Gb model
from transformers import AutoModel
model = AutoModel.from_pretrained("UFNLP/gatortron-base")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/534 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/713M [00:00<?, ?B/s]

In [4]:
## Import libraries and open CSV

def initialize_csv(csv_name):
  import csv
  import pandas as pd
  import numpy as np

  df_0 = pd.read_csv()
  df = pd.DataFrame(df_0)
  #print(df.to_string())

  col1 = "Diagnosis/Presenting problem"
  col2 = "Specific Factors 1"
  col3 = "Specific Factors 2"
  col4 = "Admitting Service"
  col5 = "Notes"
  #print(df.columns)

In [ ]:
## Parse User Input
def parse_user(prompt)
  print(prompt)
  input1 = str(input())
  #TODO: Add parsing if necessary
  return input1

# Main

In [ ]:
## Main
csv_name = 'Admission_Algorithm/Admit_table_csv.csv'
prompt1 = "What is the presenting problem? Please describe."
initialize_csv(csv_name)
input1 = parse_user(prompt1)


# Old Code

In [ ]:
#Old hard-coded method
indeces = []
for i,r in df[[col1]].iterrows():
   # print(r[0])
   #if input in r[0]:
    #   print(r[0])
    if not str(r[0]).find(input1):
        print(r[0])
        indeces.append(i)

print(indeces)
secondrow = set([])
for i in indeces:
    secondrow.add(df[col2][i])
print("Pick one of the following options:")
print(secondrow)
input2=str(input())
#for in [i]

for i,r in df[[col2]].iterrows():
   # print(r[0])
   #if input in r[0]:
    #   print(r[0])
    if i in indeces:
        if not str(r).find(input2):
            print(r)

'''
for i,r in df[[col2]].iterrows():
   # print(r[0])
   #if input in r[0]:
    #   print(r[0])
    if not str(r[0]).find(input1):

        if not str(r[1]).find(input2):
            if not str(r[2]).find(input3):
                print(r[3])
'''

# Notes

In [ ]:
#open file
#get input from user
#take input from user -> parse
#call algorithm to take two inputs: user input text and csv (can then fork to call chatGPT instead of csv in future)

#Next steps:
#Find open-source Python LLM available and easy to use to search the csv
#LLama2 etc. read which is best for our use-case
  #can use google scholar to find

#ask user's perspective on asnwer, do you agree with reccomendation, and add comments
  #store back in excel fiel with query, answer, comments -> for further